<a href="https://colab.research.google.com/github/Shavedha/Monte-Carlo-control/blob/main/RL_Ex_5_MonteCarloControl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install git+https://github.com/mimoralea/gym-walk#egg=gym-walk

  Cloning https://github.com/mimoralea/gym-walk to /tmp/pip-install-k6ow_mwx/gym-walk_2c89f0107515490a8314fbeabea38c59
  Running command git clone --filter=blob:none --quiet https://github.com/mimoralea/gym-walk /tmp/pip-install-k6ow_mwx/gym-walk_2c89f0107515490a8314fbeabea38c59
  Resolved https://github.com/mimoralea/gym-walk to commit 5999016267d6de2f5a63307fb00dfd63de319ac1
  Preparing metadata (setup.py) ... done
  Created wheel for gym-walk: filename=gym_walk-0.0.2-py3-none-any.whl size=4054 sha256=94a2c03e6b6ad92abc77fd372822fd94bbefe1b36d3ae5d9cc77572ab8f0b160
  Stored in directory: /tmp/pip-ephem-wheel-cache-xqywgspw/wheels/24/fe/c4/0cbc7511d29265bad7e28a09311db3f87f0cafba74af54d530
Successfully built gym-walk


In [2]:
import warnings ; warnings.filterwarnings('ignore')

import gym, gym_walk
import numpy as np

import random
import warnings

warnings.filterwarnings('ignore', category=DeprecationWarning)
np.set_printoptions(suppress=True)
random.seed(123); np.random.seed(123)

In [3]:
def print_policy(pi, P, action_symbols=('<', 'v', '>', '^'), n_cols=4, title='Policy:'):
    print(title)
    arrs = {k:v for k,v in enumerate(action_symbols)}
    for s in range(len(P)):
        a = pi(s)
        print("| ", end="")
        if np.all([done for action in P[s].values() for _, _, _, done in action]):
            print("".rjust(9), end=" ")
        else:
            print(str(s).zfill(2), arrs[a].rjust(6), end=" ")
        if (s + 1) % n_cols == 0:
            print("|")

In [12]:
def print_state_value_function(V, P, n_cols=4, prec=3, title='State-value function:'):
    print(title)
    for s in range(len(P)):
        v = V[s]
        print("| ", end="")
        if np.all([done for action in P[s].values() for _, _, _, done in action]):
            if title == 'State-value function:':
                print("".rjust(9), end=" ")
            else:
                print("".rjust(24), end=" ")
        else:
            print(str(s).zfill(2), '{}'.format(np.round(v, prec)).rjust(6), end=" ")
        if (s + 1) % n_cols == 0: print("|")



In [5]:
env = gym.make('FrozenLake-v1')
P = env.env.P
init_state = env.reset()
#goal_state = 6
#LEFT, RIGHT = range(2)

In [ ]:
P

Exponentially decaying schedule


In [7]:
def decay_schedule(
    init_value, min_value, decay_ratio,
    max_steps, log_start = -2, log_base=10):

    decay_steps = int(max_steps * decay_ratio)
    rem_steps = max_steps - decay_steps
    values = np.logspace(log_start,0,decay_steps,base=log_base,endpoint=True)[::-1]
    values = (values - values.min())/(values.max()-values.min())
    values = (init_value - min_value)*values+min_value
    values = np.pad(values,(0,rem_steps),'edge')
    return values



Exploratory Policy Trajectories

In [8]:
from itertools import count
def generate_trajectory(
    select_action, Q, epsilon,
    env, max_steps=200):
  done, trajectory = False, []

  while not done:
    state = env.reset()
    for t in count():
      action = select_action(state, Q, epsilon)
      next_state, reward, done, _ = env.step(action)
      experience = (state, action, reward, next_state, done)
      trajectory.append(experience)
      if done:
        break
      if t >= max_steps - 1:
        trajectory = []
        break
      state = next_state

  return np.array(trajectory, np.object)

Monte Carlo control

In [9]:
from tqdm import tqdm
def mc_control (env, gamma = 1.0,
                init_alpha = 0.5,min_alpha = 0.01, alpha_decay_ratio = 0.5,
                init_epsilon = 1.0, min_epsilon = 0.1, epsilon_decay_ratio = 0.9,
                n_episodes = 3000, max_steps = 200, first_visit = True):
  nS, nA = env.observation_space.n, env.action_space.n
  discounts = np.logspace(0,max_steps, num=max_steps,
                          base=gamma, endpoint = False)

  alphas = decay_schedule(init_alpha, min_alpha, alpha_decay_ratio, n_episodes)

  epsilons = decay_schedule(init_epsilon, min_epsilon, epsilon_decay_ratio, n_episodes)

  pi_track = []

  Q = np.zeros((nS, nA), dtype = np.float64)
  Q_track = np.zeros((n_episodes, nS, nA), dtype = np.float64)

  select_action = lambda state, Q, epsilon: np.argmax(Q[state]) if np.random.random() > epsilon_decay_ratio else np.random.randint(len(Q[state]))

  for e in tqdm(range(n_episodes), leave = False):
    trajectory = generate_trajectory(select_action, Q, epsilons[e], env, max_steps)
    visited = np.zeros((nS, nA), dtype = np.bool)
    for t, (state, action, reward, _, _) in enumerate(trajectory):
      if visited[state][action] and first_visit:
        continue
      visited[state][action] = True
      n_steps = len(trajectory[t:])
      G = np.sum(discounts[:n_steps] * trajectory[t:, 2])
      Q[state][action] += alphas[e] * (G - Q[state][action])
    Q_track[e] = Q
    pi_track.append(np.argmax(Q, axis = 1))
  V = np.max(Q, axis = 1)
  pi = lambda s: {s:a for s, a in enumerate(np.argmax(Q, axis = 1))}[s]
  return Q, V, pi

In [26]:
optimal_Q, optimal_V, optimal_pi = mc_control (env,n_episodes=150000)
print_state_value_function(optimal_Q, P, n_cols=4, prec=2, title='Action-value function:')


Action-value function:
| 00 [0.03 0.01 0.01 0.02] | 01 [0.03 0.01 0.02 0.04] | 02 [0.03 0.03 0.04 0.04] | 03 [0.02 0.03 0.02 0.03] |
| 04 [0.03 0.02 0.02 0.01] |                          | 06 [0.06 0.04 0.06 0.01] |                          |
| 08 [0.   0.05 0.06 0.07] | 09 [0.07 0.11 0.16 0.08] | 10 [0.24 0.2  0.19 0.04] |                          |
|                          | 13 [0.11 0.23 0.29 0.18] | 14 [0.24 0.57 0.57 0.47] |                          |


In [24]:
print_state_value_function(optimal_V, P, n_cols=4, prec=2, title='State-value function:')

State-value function:
| 00   0.04 | 01   0.02 | 02   0.04 | 03   0.01 |
| 04   0.04 |           | 06   0.08 |           |
| 08   0.08 | 09   0.19 | 10   0.25 |           |
|           | 13   0.23 | 14   0.56 |           |


In [25]:
print_policy(optimal_pi, P)

Policy:
| 00      < | 01      ^ | 02      < | 03      ^ |
| 04      < |           | 06      < |           |
| 08      ^ | 09      v | 10      < |           |
|           | 13      > | 14      v |           |


In [20]:
def probability_success(env, pi, goal_state, n_episodes=100, max_steps=200):
    random.seed(123); np.random.seed(123) ; env.seed(123)
    results = []
    for _ in range(n_episodes):
        state, done, steps = env.reset(), False, 0
        while not done and steps < max_steps:
            state, _, done, h = env.step(pi(state))
            steps += 1
        results.append(state == goal_state)
    return np.sum(results)/len(results)

In [21]:
def mean_return(env, pi, n_episodes=100, max_steps=200):
    random.seed(123); np.random.seed(123) ; env.seed(123)
    results = []
    for _ in range(n_episodes):
        state, done, steps = env.reset(), False, 0
        results.append(0.0)
        while not done and steps < max_steps:
            state, reward, done, _ = env.step(pi(state))
            results[-1] += reward
            steps += 1
    return np.mean(results)

In [22]:
def results(env,optimal_pi,optimal_V,P):
    print('Reaches goal {:.2f}%. Obtains an average undiscounted return of {:.4f}.'.format(
    probability_success(env, optimal_pi, goal_state=goal_state)*100,
    mean_return(env, optimal_pi)))

In [23]:
goal_state = 15
results(env,optimal_pi,optimal_V,P)

Reaches goal 66.00%. Obtains an average undiscounted return of 0.6600.
